In [126]:
import pandas as pd
import numpy as np



##Preprocessing

In [127]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [128]:
anxiety = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_Anxiety.csv")
# anxiety

In [129]:
anxiety.columns

Index(['title', 'selftext'], dtype='object')

###With Title

In [130]:
# anxiety = (anxiety["title"] + ". " + anxiety["selftext"]).values
# depression = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_depression.csv")
# depression = (depression["title"] + ". " + depression["selftext"]).values
# rndm = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_non_depression.csv")
# rndm = (rndm["title"] + ". " + rndm["selftext"]).values

### Without Title

In [131]:
anxiety = anxiety["selftext"].values

In [132]:
depression = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_depression.csv")

In [133]:
depression = (depression["selftext"]).values

In [134]:
rndm = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_non_depression.csv")

In [135]:
rndm = (rndm["selftext"]).values

In [136]:
anxiety.shape, depression.shape, rndm.shape

((1846,), (1827,), (2408,))

In [137]:
anxiety = np.random.choice(anxiety, 1827)
depression = np.random.choice(depression, 1827)
rndm = np.random.choice(rndm, 1827)

In [138]:
x = np.concatenate([depression, rndm])
y_depression = np.zeros((len(depression), 2))
for i in range(len(depression)):
  y_depression[i][0] = 1.0

y_nonDepression = np.zeros((len(rndm), 2))
for i in range(len(rndm)):
  y_nonDepression[i][1] = 1.0

y_anxiety = np.zeros((len(anxiety), 2))
for i in range(len(anxiety)):
  y_anxiety[i][0] = 1.0

y = np.concatenate([y_depression, y_nonDepression])

In [139]:
print(y)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [140]:
x.shape, y.shape

((3654,), (3654, 2))

In [141]:
from sklearn.model_selection import train_test_split
train, test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [142]:
len(train), len(y_train)

(2448, 2448)

In [143]:
print(y_train)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]


##Embedding

In [144]:
!pip install -q datasets

In [145]:
!pip install -q transformers

In [146]:
!pip install -q accelerate -U # YOU MUST RESTART YOUR RUNTIME AFTER INSTALLING

In [147]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokens, tokens_test, anxiety_tokens = [], [], []

for text,y in zip(train, y_train):
    emb = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    emb["labels"] = y
    tokens.append(emb)

for text_test, y in zip(test, y_test):
  emb = tokenizer(text_test, padding="max_length", truncation=True, max_length=128)
  emb["labels"] = y
  tokens_test.append(emb)

for anx, y in zip(anxiety, y_anxiety):
  emb = tokenizer(anx, padding="max_length", truncation=True, max_length=128)
  emb["labels"] = y
  anxiety_tokens.append(emb)

train_data = {"input": tokens, "labels": y_train}
test_data = {"input": tokens_test, "labels": y_test}
anxiety_data = {"input": anxiety_tokens, "labels": np.ones(len(anxiety_tokens))}

In [148]:
print(anxiety_data['input'][0])

{'input_ids': [101, 3383, 2023, 2003, 1037, 2033, 3277, 1010, 2021, 1045, 2031, 2467, 2042, 2428, 19810, 2000, 2202, 4620, 1999, 2270, 1045, 2467, 4737, 2008, 1045, 2298, 10021, 2635, 4620, 1997, 2242, 2066, 29132, 2006, 1996, 2723, 2138, 2000, 2033, 2009, 3504, 5875, 1045, 2113, 2009, 1521, 1055, 2026, 10089, 2893, 1999, 1996, 2126, 1010, 1998, 1996, 2111, 2105, 2033, 2097, 3497, 2196, 2156, 2033, 2153, 1010, 2664, 1045, 2145, 5998, 2007, 2122, 5346, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [149]:
from datasets import Dataset
train_set = Dataset.from_dict(train_data)
test_set = Dataset.from_dict(test_data)
anxiety_set = Dataset.from_dict(anxiety_data)

In [150]:
print(train_set["labels"])


[[0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.

In [151]:
tokenizer.decode(train_set["input"][0]['input_ids'])

"[CLS] for the past 40 years, nearly every single championship team has had at least one player that was * * * a 1st team all - nba selection in years prior to that title year * * * i could only find 2 exceptions to this rule : 1 steph curry's first selection to 1st team all - nba was the year he won his first title 2 the 04 ’ pistons did not field a player ever selected that list for every other year, the champs had at least one such player in nearly 50 % of those years, the champ fielded 2 such players * * so, who meets the criteria this year * * 1 76 [SEP]"

In [152]:
train_set.set_format("torch")
test_set.set_format("torch")
anxiety_set.set_format("torch")

Ywo label types: negative feeling or not

In [153]:
# id2label = {0.0: "NEGATIVE", 1.0: "POSITIVE"}
# label2id = {"NEGATIVE": 0.0, "POSITIVE": 1.0}
id2label = {0: "NEGATIVE", 1:'NON-NEGATIVE'}
label2id = {"NEGATIVE": 0, 'NON-NEGATIVE': 1}

In [154]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##Training

In [155]:
metric_name = "f1"
batch_size = 16

In [156]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [157]:
# REFERENCE CODE

In [158]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from transformers import EvalPrediction
import torch

# def metrics(x, y_true, threshold=0.5):
#   sigmoid = torch.nn.Sigmoid()
#   probs = sigmoid(torch.Tensor(x))
#   y_pred = np.zeros(probs.shape)
#   y_pred[np.where(probs >= threshold)] = 1
#   precision = precision_score(y_true, y_pred)
#   recall = recall_score(y_true, y_pred)
#   f1 = f1_score(y_true, y_pred)
#   print( "Precision: ", precision, ". Recall: ", recall, ". F1: ", f1)


# def compute_metrics(p: EvalPrediction):
#     preds = p.predictions[0] if isinstance(p.predictions,
#             tuple) else p.predictions

#     metrics(predictions=preds, labels=p.label_ids)
#     # return result

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average = 'micro')
    recall = recall_score(y_true, y_pred, average = 'micro')
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'precision': precision,
               'recall': recall}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [159]:
train_set['input'][0]['labels'].type()

'torch.FloatTensor'

In [160]:
train_set['input'][0]['input_ids'].size()

torch.Size([128])

In [161]:
train_set['input'][0]['input_ids'].unsqueeze(0).size()

torch.Size([1, 128])

In [162]:
print(train_set['input'][0].keys())

dict_keys(['attention_mask', 'input_ids', 'labels', 'token_type_ids'])


In [163]:
outputs = model(input_ids=train_set["input"][0]['input_ids'].unsqueeze(0), labels=train_set['input'][0]['labels'].unsqueeze(0))
outputs


SequenceClassifierOutput(loss=tensor(0.8751, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.2594, -0.4099]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

### Train

In [165]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_set['input'],
    eval_dataset=anxiety_data['input'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [166]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy,Precision,Recall
1,No log,0.259353,0.905660,0.905583,0.903120,0.904918,0.906404
2,No log,0.514415,0.807279,0.808703,0.801314,0.813333,0.801314
3,No log,0.552041,0.815486,0.816092,0.812808,0.818182,0.812808


TrainOutput(global_step=459, training_loss=0.1694396108323972, metrics={'train_runtime': 232.8756, 'train_samples_per_second': 31.536, 'train_steps_per_second': 1.971, 'total_flos': 483071897640960.0, 'train_loss': 0.1694396108323972, 'epoch': 3.0})

In [167]:
trainer.evaluate()

{'eval_loss': 0.2593529174085472,
 'eval_f1': 0.9056603773584905,
 'eval_roc_auc': 0.9055829228243021,
 'eval_accuracy': 0.90311986863711,
 'eval_precision': 0.9049180327868852,
 'eval_recall': 0.9064039408866995,
 'eval_runtime': 13.5708,
 'eval_samples_per_second': 134.628,
 'eval_steps_per_second': 8.474,
 'epoch': 3.0}